0. Указываем необходимые константы:

In [0]:
IP_ADDRESS = ''

USERNAME = ''
PASSWORD = ''
LOCAL_PORT = 22
REMOTE_PORT = 2222
ROOT_PASSWORD = None 

1. Запускаем ssh-сервер и даём доступ в контейнер пользователю root

In [0]:
#Generate root password
import secrets, string
if not(type(ROOT_PASSWORD) == str):
  ROOT_PASSWORD = ''.join(secrets.choice(string.ascii_letters + string.digits) for i in range(20))

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen sshpass > /dev/null
#Set root password
! echo root:$ROOT_PASSWORD | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Print root password
print("Root password on CoLab: {}".format(ROOT_PASSWORD))

2. Загружаем список доступных портов для туннелирования

In [0]:
import codecs
import csv
from urllib.request import urlopen


def parse_ports(ports):
  for port in ports:
    if ':' in port:
      yield tuple(map(int, port.split(':')))
    else:
      yield int(port)


allowed_ports = {}
      
with urlopen(f'http://{IP_ADDRESS}:1111/ports.csv') as response:
  for (user, *ports) in csv.reader(codecs.iterdecode(response, 'utf8')):
    allowed_ports[user] = list(parse_ports(ports))

3. Создаём функцию для выполнения туннелирования

In [0]:
import os
import signal
import subprocess
import time


def check_port(username, port):
  if username not in allowed_ports:
    return False
  
  for item in allowed_ports[username]:
    if isinstance(item, int) and item == port:
      return True
    if isinstance(item, tuple) and port in range(*item):
      return True
    
  return False


def get_ports(username):
  ports = []
  
  for item in allowed_ports.get(username, []):
    if isinstance(item, int):
      ports.append(str(item))
    elif isinstance(item, tuple):
      ports.append(f'{item[0]}..{item[1]}')
      
  return ', '.join(ports)


def check_tunnel_up(remote_port):
  ps_output = subprocess.check_output(['ps', 'ax'], encoding='utf8')
  
  for line in ps_output.split('\n'):
    if 'sshpass' in line:
      return False
    
    if ('ssh -o' in line and f'{IP_ADDRESS}:{remote_port}' in line and
        'sshpass' not in line):
      return int(line.split()[0])
    
  return False


def forward_port(local_port, remote_port, username, password):
  if not check_port(username, remote_port):
    print(f'ERROR: Remote port {remote_port} is not allowed for user "{username}"')
    print(f'Allowed ports is: {get_ports(username)}')
    return False
  
  if 'ROOT_PASSWORD' not in globals():
    print('ERROR: No root password')
    return False
  
  if not password:
    print('ERROR: empty password')
    return False
  
  pid = check_tunnel_up(remote_port)
  if pid:
    print(f'ERROR: Tunnel is up. Kill tunnel by command: os.kill({pid}, signal.SIGKILL)')
    return False
  
  print(f'Creating tunnel localhost:{local_port} <--> {IP_ADDRESS}:{remote_port}')
  
  cmd = (f'sshpass -p "{password}" ssh -o "StrictHostKeyChecking=no" -fN -R '
      f'"{IP_ADDRESS}:{remote_port}:localhost:{local_port}" '
      f'"{username}@{IP_ADDRESS}" &')
  
  print('DEBUG: ', cmd)
  
  # Run reverse ssh tunnel
  get_ipython().system_raw(cmd)
  
  time.sleep(1)
  
  for i in range(10):
    if check_tunnel_up(remote_port):
      print('Success')
      return True
    
    time.sleep(1)
  else:
    print('Unknown error on creating tunnel')
    return False

4. Создаём туннель. 

Если нужно пробросить несколько портов, скопируйте код ниже несколько раз, указав в каждой копии необходимые порты.

In [0]:
if forward_port(LOCAL_PORT, REMOTE_PORT, USERNAME, PASSWORD):
  print(f'\nCommand for CoLab access: `ssh -p {REMOTE_PORT} root@{IP_ADDRESS}`'
        f'\nPassword: "{ROOT_PASSWORD}"')

In [0]:
!ps ax | grep ssh